In [ ]:
# !pip install konlpy
# !pip install krwordrank

In [1]:
from konlpy.tag import Okt
import pandas as pd
import numpy as np
import re
import requests, bs4
from urllib.parse import unquote

# 텍스트 요약
from krwordrank.sentence import summarize_with_sentences
from krwordrank.word import KRWordRank
from krwordrank.word import summarize_with_keywords

### 강소기업 정보/탐방기 - 워크넷 API
- 'busiNo': 사업자 등록번호
- 'compNm': 사업자명
- 'regionNm': 지역
- 'indTypeNm': 업종명
- 'smlgntCoClcd': 강소기업 카테고리

In [ ]:
My_API_Key = unquote('#########')  # 본인 인증키

# 추출 컬럼명
data_idx = ['busiNo', 'compNm', 'regionNm', 'indTypeNm', 'smlgntCoClcd']
value_idx = [[] for _ in range(len(data_idx))]
data = dict(zip(data_idx, value_idx)) # 각 컬럼 초기화

In [ ]:
# 1페이지 ~ 1000페이지 탐색
for page in range(1, 1001):
    # Service URL + 인증키
    xmlUrl = f'''http://openapi.work.go.kr/opi/opi/opia/corpInfoReportApi.do?authKey={My_API_Key}\
            &returnType=XML&callTp=L&startPage={page}&display=100'''

    # Get > parsing
    response = requests.get(xmlUrl).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
    wanted = xmlobj.find_all("company")
    if wanted == []:
        break
    
    # 강소기업 정보 parsing
    for vals in wanted:
        data["busiNo"].append(vals.find('busiNo').text) # 사업자등록번호
        data["compNm"].append(vals.find('compNm').text) # 사업자명

        try:
            data["regionNm"].append(vals.find('regionNm').text) # 지역명
        except AttributeError:
            data["regionNm"].append(np.nan)
            #print("regionNm >> ", vals)

        try:
            data["smlgntCoClcd"].append(vals.find('smlgntCoClcd').text) # 강소기업 카테고리
        except AttributeError:
            data["smlgntCoClcd"].append(np.nan)
            #print("smlgntCoClcd >> ", vals)

        data["indTypeNm"].append(vals.find('indTypeNm').text) # 업종명

In [ ]:
# 강소기업 정보
data = pd.DataFrame(data)

In [ ]:
# 강소기업 탐방기 parsing
QA_data = {'busiNo':[], 'compNm':[], 'Q':[], 'A':[]}
for row in data.iterrows():
    busiNO = row[1]['busiNo']
    comp = row[1]['compNm']

    xmlUrl = f'''http://openapi.work.go.kr/opi/opi/opia/corpInfoReportApi.do?authKey={My_API_Key}\
                &returnType=XML&callTp=D&busiNo={busiNO}'''

    response = requests.get(xmlUrl).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
    QAs = xmlobj.find("compDtl").find_all('coInqReprt') # Q&A 정보
    for qa in QAs:
        QA_data['busiNo'] += [busiNO] # 사업자등록번호
        QA_data['compNm'] += [comp] # 기업명
        QA_data['Q'] += [qa.find("contSubtitle").text] # 질문
        QA_data['A'] += [qa.find("contCont").text] # 답변

In [3]:
QA_data = pd.DataFrame(QA_data)
QA_data.head(2)

,busiNo,compNm,Q,A
0,3058140342,(주)예람,회사의 주력 분야 등 간단한 회사 소개를 해주세요.,1999년도에 창립하여 19년째 운영중인 ㈜예람의 주력 사업 분야 중 첫 번째는 입...
1,3058140342,(주)예람,회사의 장기적인 목표나 경영 철학이 있으신가요?,"경영 철학이자 회사의 4대 이념으로 가치, 인정, 나눔, 신용이 있습니다. \r\n..."


In [ ]:
QA_data.to_csv("../data/탐방기업_QA.csv", index=False, encoding = 'UTF-8-sig')
QA_data.to_excel("../data/탐방기업_QA.xlsx", index=False, encoding = 'UTF-8-sig')

### Q&A 단어/어절 분리

In [4]:
# engine='python': csv 파일 한글명 읽기 위해
QA = pd.read_csv("../data/탐방기업_QA.csv", engine='python', encoding = 'utf-8-sig')
QA.head()

,busiNo,compNm,Q,A
0,3058140342,(주)예람,회사의 주력 분야 등 간단한 회사 소개를 해주세요.,1999년도에 창립하여 19년째 운영중인 ㈜예람의 주력 사업 분야 중 첫 번째는 입...
1,3058140342,(주)예람,회사의 장기적인 목표나 경영 철학이 있으신가요?,"경영 철학이자 회사의 4대 이념으로 가치, 인정, 나눔, 신용이 있습니다. \r\n..."
2,3058140342,(주)예람,타사와 비교했을 때 귀사의 장점은 무엇인가요?,직원의 성장을 위해 전폭적인 지원을 합니다. 업무관련 역량을 위한 내/외부 교육지원...
3,3058140342,(주)예람,회사의 채용절차 및 선호하는 인재상은 어떻게 되나요?,"주로 배움에 대한 열정, 올바른 인성을 갖춘 인재를 선호합니다. 채용절차는 1차 서..."
4,3058140342,(주)예람,신입 평균 급여수준 및 인사 제도 현황에 대해 알 수 있나요?,"신입 평균 급여는 올해 2,200만원입니다. 연봉 외에는 부서 성과에 따른 성과급 ..."


In [ ]:
# 답변에 대한 단어/어절 분리
okt = Okt()
for row in QA.iterrows():
    try:
        nouns = okt.nouns(row[1]['A'])
        QA.at[row[0], 'nouns'] = ','.join(nouns)
        QA.at[row[0], 'phrases'] = ','.join(okt.phrases(row[1]['A']))
    except AssertionError:
        QA.at[row[0], 'nouns'] = np.nan
        QA.at[row[0], 'phrases'] = np.nan

QA = QA.dropna().reset_index(drop='index')
QA.to_csv("QA_tokenData.csv", index=False, encoding = 'utf-8-sig')

### Q&A 텍스트 요약

In [ ]:
okt = Okt()

# 데이터 전처리
def preprocessing(review):
    total_review = ''

    #인풋리뷰
    for row in review.iterrows():
        r = row[1]['A']
        #하나의 리뷰에서 문장 단위로 자르기
        sentence = r
        sentence = re.sub('\s','',sentence)
        # sentence = re.sub('\u200b','',sentence)
        # sentence = re.sub('\xa0','',sentence)
        sentence = re.sub('([a-zA-Z])','',sentence)
        sentence = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',sentence)
        sentence = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',sentence)

        if len(sentence) == 0:
            continue
        sentence = okt.phrases(sentence)
        
        word = []
        for i in sentence:
            if i == '디자':
                i = '디자인'
            elif i == '시스':
                i = '시스템'
            elif i == '네트':
                i = '네트워크'
            word.append(i)
        word = ' '.join(word)
        word += '. '
        total_review += word
    return total_review

In [ ]:
QA2 = QA.iloc[np.where(QA['Q'].str.contains('복리|복지'))] # 복리/복지 관련 질문에 대한 답변 모음

In [ ]:
token_data = {'busiNo':[], 'company':[], 'tag':[]}

# 키워드 추출
for b_idx, comp_qa in list(QA2.groupby("busiNo")):
    print(b_idx)
    st = ''
    name = comp_qa['compNm'].iloc[0]
    texts = preprocessing(comp_qa)
    st += texts
    texts = st.split('. ')

    # 불용어
    stopwords = {"있습니다.", "또한", '저희', '회사', '대한', '때문', '통해', 
                '경우', '위해', '각종', '직원의', '업무', '직원들', '당사', '충족',
                '후생','황금','화성의','현지','현재','허가'}
    try:
        keywords, sents = summarize_with_sentences(
                                               texts, 
                                               stopwords = stopwords,
                                               num_keywords=100, 
                                               num_keysents=10
                                               )
    except ValueError:
        keywords, sents = summarize_with_sentences(
                                               texts*5, 
                                               stopwords = stopwords,
                                               num_keywords=100, 
                                               num_keysents=10
                                               )
        
    for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:7]:
        print('#%s' % word)
        token_data['busiNo'].append(b_idx)
        token_data['company'].append(name)
        token_data['tag'].append(word)

    print()

In [5]:
token_df = pd.DataFrame(token_data)
token_df.head(2)

,busiNo,company,tag
0,1018183356,(주)샤뽀,퇴근
1,1018183356,(주)샤뽀,정시


In [ ]:
token_df.to_csv("../data/QA_tag4_3.csv", index=False, encoding = 'utf-8-sig')

#### 복지 단어 개수 count

In [6]:
data = pd.read_csv("../data/QA_tag4_3.csv")
data.head(5)

,busiNo,company,tag
0,1018183356,(주)샤뽀,퇴근
1,1018183356,(주)샤뽀,정시
2,1018183356,(주)샤뽀,기준
3,1018183356,(주)샤뽀,준수
4,1018183356,(주)샤뽀,상황


In [7]:
data1 = pd.DataFrame(data.value_counts(subset=['company']))
data1 = data1.reset_index()
data1.columns = ['company', 'num']
data1.head()

,company,num
0,(주)성창,10
1,(주)엠알티,7
2,(주)엠에코,7
3,(주)엠코전자,7
4,(주)연한훼미리,7


In [8]:
data1.to_csv("../data/복지_기업별단어수.csv", index=False, encoding="cp949")

#### 뉴스 크롤링 및 뉴스 기사 개수 count

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
data_idx = ['keyword', 'title']
value_idx = [[] for _ in range(len(data_idx))]
data = dict(zip(data_idx, value_idx)) # 각 컬럼 초기화

In [13]:
# 기업명 가져오기
name = pd.read_excel("../data/탐방기업_QA.xlsx")
names = name[['compNm']].drop_duplicates(ignore_index = True)
names.head()

,compNm
0,(주)예람
1,(주)티아이
2,(주)에스에스알
3,(주)보템
4,(주)제이오텍


In [ ]:
# 기업 별 뉴스 기사 카운팅
for row in names.iterrows():
    keyword = row[1]['compNm']
    lastPage = 10
    pageNum = 1
    for i in range(1, int(lastPage) * 10, 10):
        print(pageNum)
        response = requests.get(f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query={keyword}&start={i}")
        html = response.text

        soup = BeautifulSoup(html, 'html.parser')
        links = soup.select(".news_tit") #결과가 리스트로 나옴
        #print(links)

        for link in links:
            title = link.text # 태그 안에 텍스트요소를 가져온다.
            url = link.attrs['href'] # href의 속성값을 가져온다.
            # print(title)
            # print(url)
            data['title'].append(title)
            # data['url'].append(url)
            data['keyword'].append(keyword)


        pageNum = pageNum + 1

In [ ]:
data = pd.DataFrame(data)
data.to_csv("../data/뉴스검색수.csv", index=False, encoding = 'UTF-8-sig')

In [ ]:
data1 = pd.DataFrame(data.value_counts(subset=['keyword']))
data1 = data1.reset_index()
data1.columns = ['company', 'num']
data1.head()

In [ ]:
data1.to_csv("../data/뉴스_기업별단어수.csv", index=False, encoding="cp949")

#### 국민연금데이터와 기업탐방 데이터 병합

In [14]:
# 기업탐방 데이터
data1 = pd.read_csv("../data/탐방기업_QA.csv")
data1['name'] = data1['compNm']
data1['name'].replace("주식회사", '', regex=True, inplace=True)
data1['name'].replace("\(주\)", '', regex=True, inplace=True)
data1['name'].replace(" ", '', regex=True, inplace=True)
data1.head(2)

,busiNo,compNm,Q,A,name
0,3058140342,(주)예람,회사의 주력 분야 등 간단한 회사 소개를 해주세요.,1999년도에 창립하여 19년째 운영중인 ㈜예람의 주력 사업 분야 중 첫 번째는 입...,예람
1,3058140342,(주)예람,회사의 장기적인 목표나 경영 철학이 있으신가요?,"경영 철학이자 회사의 4대 이념으로 가치, 인정, 나눔, 신용이 있습니다. \r\n...",예람


In [15]:
name = data1[['name']].drop_duplicates()

In [ ]:
# 국민연금 데이터
data = pd.read_csv("../data/국민연금공단_국민연금 가입 사업장 내역_20220121.csv")

data['퇴직률'] = round((data['상실가입자수']/data['가입자수'])*100, 2)
data['평균급여'] = round((data['당월고지금액']/(data['가입자수']*0.09)), 2) 

# 강소기업 리스트와 merge하기 위해 사업장명 통일
data['회사이름'] = data['사업장명']
data['회사이름'].replace("주식회사", '', regex=True, inplace=True)
data['회사이름'].replace("\(주\)", '', regex=True, inplace=True)
data['회사이름'].replace(" ", '', regex=True, inplace=True)

In [17]:
data2 = data[['사업장명', '사업자등록번호', '사업장업종코드명', 
            '가입자수', '상실가입자수', '퇴직률', '회사이름', '평균급여']]
data2.head(1)

,사업장명,사업자등록번호,사업장업종코드명,가입자수,상실가입자수,퇴직률,회사이름,평균급여
0,대신정보통신(주),133850,응용 소프트웨어 개발 및 공급업,551,44,7.99,대신정보통신,3534767.09


In [18]:
# 기업탐방기와 국민연금 데이터를 회사이름으로 겹치는 기업만 병합
df_LEFT_JOIN = pd.merge(name, data2, left_on='name', right_on='회사이름', how='inner')
df_LEFT_JOIN.head(1)
df_LEFT_JOIN = df_LEFT_JOIN[df_LEFT_JOIN['사업장명'] != ' ']
tmp = df_LEFT_JOIN[['사업장명', '사업자등록번호', '사업장업종코드명', '가입자수', '상실가입자수', '퇴직률', '평균급여']]
tmp.head(2)

,사업장명,사업자등록번호,사업장업종코드명,가입자수,상실가입자수,퇴직률,평균급여
0,(주)예람,305814,기타 정보 기술 및 컴퓨터 운영 관련 서비스업,63,0,0.0,2673079.37
1,(주)예람,301816,도배 실내 장식 및 내장 목공사업,3,0,0.0,2360296.30


In [ ]:
tmp.to_excel("../data/강소기업_급여및퇴직률.xlsx", index=False)

#### 최종 방사형 데이터 생성

In [20]:
data1 = pd.read_excel("../data/강소기업_급여및퇴직률.xlsx")
data2 = pd.read_csv("../data/복지_기업별단어수.csv", encoding='cp949')
data3 = pd.read_csv("../data/뉴스_기업별단어수.csv", encoding='cp949')

data2.columns = ['사업장명', '복지']
data3.columns = ['사업장명', '뉴스']
data2.head()

,사업장명,복지
0,(주)성창,10
1,(주)엠알티,7
2,(주)엠에코,7
3,(주)엠코전자,7
4,(주)연한훼미리,7


In [21]:
# 사업장명 통일
data1['회사이름'] = data1['사업장명']
data1['회사이름'].replace("주식회사", '', regex=True, inplace=True)
data1['회사이름'].replace("\(주\)", '', regex=True, inplace=True)
data1['회사이름'].replace(" ", '', regex=True, inplace=True)

data2['회사이름'] = data2['사업장명']
data2['회사이름'].replace("주식회사", '', regex=True, inplace=True)
data2['회사이름'].replace("\(주\)", '', regex=True, inplace=True)
data2['회사이름'].replace(" ", '', regex=True, inplace=True)

data3['회사이름'] = data3['사업장명']
data3['회사이름'].replace("주식회사", '', regex=True, inplace=True)
data3['회사이름'].replace("\(주\)", '', regex=True, inplace=True)
data3['회사이름'].replace(" ", '', regex=True, inplace=True)

In [22]:
# 데이터 세가지 병합
merge_data = pd.merge(data1, data2[['복지', '회사이름']], left_on='회사이름', right_on='회사이름', how = 'outer')
merge_data2 = pd.merge(merge_data, data3[['뉴스', '회사이름']], left_on='회사이름', right_on='회사이름', how = 'outer')
merge_data2 = merge_data2.dropna(subset=['사업장명']).drop(['회사이름'], axis=1)
merge_data2.tail()

,사업장명,사업자등록번호,사업장업종코드명,가입자수,상실가입자수,퇴직률,평균급여,복지,뉴스
1039,주식회사케이엠테크,805810.0,기타 화학 물질 및 화학제품 도매업,3.0,0.0,0.00,3082962.96,1.0,1.0
1040,(주)케이엠테크,779860.0,그 외 기타 플라스틱 제품 제조업,5.0,0.0,0.00,2179777.78,1.0,1.0
1041,주식회사케이엠테크,286860.0,의료용품 도매업,6.0,0.0,0.00,2776296.30,1.0,1.0
1042,(주)미쓰윤,214884.0,광고 대행업,49.0,0.0,0.00,3028480.73,7.0,12.0
1043,유양기술(주),125812.0,산업용 난방보일러 및 방열기 제조업,83.0,3.0,3.61,3854358.77,7.0,19.0


In [24]:
merge_data2.isna().sum()

사업장명         0
사업자등록번호      0
사업장업종코드명    18
가입자수         0
상실가입자수       0
퇴직률          1
평균급여         4
복지          68
뉴스          25
dtype: int64

In [25]:
# 복지 단어 개수가 카운트 되지 않은 기업 -> 0으로 초기화
merge_data2['복지'] = merge_data2['복지'].apply(lambda x : 0 if str(x) == 'nan' else x)

# 뉴스 단어 개수가 카운트 되지 않은 기업 -> 0으로 초기화
merge_data2['뉴스'] = merge_data2['뉴스'].apply(lambda x : 0 if str(x) == 'nan' else x)

print(merge_data2['복지'].isnull().sum(), merge_data2['뉴스'].isnull().sum())

0 0


In [26]:
# 퇴직률이 낮을수록 높은 등급을 주기 위해 100에서 빼기 수행
merge_data2['퇴직률'] = 100 - merge_data2['퇴직률']

# 가입자가 없는 경우(분모가 0), 퇴직률 0으로 초기화
merge_data2['퇴직률'] = merge_data2['퇴직률'].apply(lambda x : 0 if str(x) == '-inf' else x)
merge_data2['퇴직률'] = merge_data2['퇴직률'].apply(lambda x : 0 if str(x) == 'inf' else x)
merge_data2['퇴직률'] = merge_data2['퇴직률'].apply(lambda x : 0 if str(x) == 'nan' else x)

merge_data2 = merge_data2.astype({"퇴직률":int})

In [27]:
# 가입자 수에 따른 등급 부여 -> 한 등급 당 동일 개수의 회사
merge_data2['가입자수'] = pd.qcut(merge_data2['가입자수'], 5, labels = list(range(1, 6)))

# 퇴직률, 평균급여, 복지, 뉴스 등급 부여 -> 구간 별 등급 부여
merge_data2['퇴직률'] = pd.cut(merge_data2['퇴직률'], 5, labels = list(range(1, 6)))
merge_data2['평균급여'] = pd.cut(merge_data2['평균급여'], 5, labels = list(range(1, 6)))
merge_data2['복지'] = pd.cut(merge_data2['복지'], 5, labels = list(range(1, 6)))
merge_data2['뉴스'] = pd.cut(merge_data2['뉴스'], 5, labels = list(range(1, 6)))
merge_data2.head()

,사업장명,사업자등록번호,사업장업종코드명,가입자수,상실가입자수,퇴직률,평균급여,복지,뉴스
0,(주)예람,305814.0,기타 정보 기술 및 컴퓨터 운영 관련 서비스업,5,0.0,5,3,3,2
1,(주)예람,301816.0,도배 실내 장식 및 내장 목공사업,1,0.0,5,2,3,2
2,주식회사예람,130864.0,장례식장 및 장의관련 서비스업,1,0.0,5,1,3,2
3,(주)티아이,513814.0,기타 반도체 소자 제조업,4,2.0,5,4,4,5
4,(주)티아이,137819.0,기타 반도체 소자 제조업,1,0.0,5,2,4,5


In [28]:
# 태블로에서 방사형 데이터를 그리기 위해 데이터 melting
cut_data = pd.melt(merge_data2.drop(columns=['상실가입자수']), id_vars=['사업장명', '사업자등록번호', '사업장업종코드명'])
cut_data.columns = ['사업장명',	'사업자등록번호',	'사업장업종코드명',	'항목',	'등급']
cut_data.head()

,사업장명,사업자등록번호,사업장업종코드명,항목,등급
0,(주)예람,305814.0,기타 정보 기술 및 컴퓨터 운영 관련 서비스업,가입자수,5
1,(주)예람,301816.0,도배 실내 장식 및 내장 목공사업,가입자수,1
2,주식회사예람,130864.0,장례식장 및 장의관련 서비스업,가입자수,1
3,(주)티아이,513814.0,기타 반도체 소자 제조업,가입자수,4
4,(주)티아이,137819.0,기타 반도체 소자 제조업,가입자수,1


In [ ]:
cut_data.to_excel("../data/방사형데이터_구간화.xlsx", index=False, encoding = 'UTF-8-sig')

#### 인터뷰 내용 추가

In [29]:
interview = pd.read_csv("../data/탐방기업_QA.csv")
interview['회사이름'] = interview['compNm']
interview['회사이름'].replace("주식회사", '', regex=True, inplace=True)
interview['회사이름'].replace("\(주\)", '', regex=True, inplace=True)
interview['회사이름'].replace(" ", '', regex=True, inplace=True)
interview.head()

,busiNo,compNm,Q,A,회사이름
0,3058140342,(주)예람,회사의 주력 분야 등 간단한 회사 소개를 해주세요.,1999년도에 창립하여 19년째 운영중인 ㈜예람의 주력 사업 분야 중 첫 번째는 입...,예람
1,3058140342,(주)예람,회사의 장기적인 목표나 경영 철학이 있으신가요?,"경영 철학이자 회사의 4대 이념으로 가치, 인정, 나눔, 신용이 있습니다. \r\n...",예람
2,3058140342,(주)예람,타사와 비교했을 때 귀사의 장점은 무엇인가요?,직원의 성장을 위해 전폭적인 지원을 합니다. 업무관련 역량을 위한 내/외부 교육지원...,예람
3,3058140342,(주)예람,회사의 채용절차 및 선호하는 인재상은 어떻게 되나요?,"주로 배움에 대한 열정, 올바른 인성을 갖춘 인재를 선호합니다. 채용절차는 1차 서...",예람
4,3058140342,(주)예람,신입 평균 급여수준 및 인사 제도 현황에 대해 알 수 있나요?,"신입 평균 급여는 올해 2,200만원입니다. 연봉 외에는 부서 성과에 따른 성과급 ...",예람


In [ ]:
# 첫번째 질문만
interview.drop_duplicates(subset = ['busiNo', 'compNm'], keep='first', ignore_index=True, inplace=True)

In [ ]:
radar_data = cut_data

for row in interview.iterrows():
    busiNo = int(str(row[1]['busiNo'])[:6])
    comp = row[1]['회사이름']

    # 인터뷰한 기업의 등급 정보
    interv_comp = radar_data[(radar_data['회사이름'] == comp)&(radar_data['사업자등록번호'] == busiNo)]
    for vals in interv_comp.iterrows():
        idx = vals[0] # index
        if '소개' in row[1]['Q']:
            radar_data.at[idx, 'Q'] = '기업 정보'
        else:
            radar_data.at[idx, 'Q'] = row[1]['Q']
            # print(row[1]['Q'])
        radar_data.at[idx, 'A'] = row[1]['A'].replace("\r\n", " ")

radar_data = radar_data.dropna().reset_index(drop=True)

In [ ]:
radar_data.to_excel("../data/방사형데이터_인터뷰추가.xlsx", index=False)